# Fine-Tuning MobileBERT with Federated Averaging

In [1]:
# Copyright 2020, The TensorFlow Federated Authors.
# Copyright 2020, Ronald Seoh
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Google Colab settings

In [2]:
# Use Google Colab
use_colab = True

# Is this notebook running on Colab?
# If so, then google.colab package (github.com/googlecolab/colabtools)
# should be available in this environment

# Previous version used importlib, but we could do the same thing with
# just attempting to import google.colab
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    drive.mount('/content/drive')
    
    # If there's a package I need to install separately, do it here
    !pip install pyro-ppl

    # cd to the appropriate working directory under my Google Drive
    %cd 'drive/My Drive/Colab Notebooks/BERTerated'
    
    # List the directory contents
    !ls

# IPython reloading magic
%load_ext autoreload
%autoreload 2

# Random seeds
# Based on https://pytorch.org/docs/stable/notes/randomness.html
random_seed = 692

In [3]:
# Install required packages
!pip install tensorflow-federated-nightly==0.16.1.dev20201021 transformers==3.4.0

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


## Import packages

In [4]:
import sys
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import transformers

import nest_asyncio
nest_asyncio.apply()

import simple_fedavg_tf
import simple_fedavg_tff

# Random seed settings
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

# Test the TFF is working:
tff.federated_computation(lambda: 'Hello, World!')()

/Users/se9/miniconda3/envs/idp-tensorflow-federated/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:43: UserWarning: You are currently using a nightly version of TensorFlow (2.5.0-dev20201108). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  UserWarning,


b'Hello, World!'

In [5]:
# Print version information
print("Python version: " + sys.version)
print("NumPy version: " + np.__version__)
print("TensorFlow version: " + tf.__version__)
print("TensorFlow Federated version: " + tff.__version__)
print("Transformers version: " + transformers.__version__)

Python version: 3.7.7 (default, Jun 26 2020, 02:44:13) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
NumPy version: 1.18.5
TensorFlow version: 2.5.0-dev20201108
TensorFlow Federated version: 0.16.1
Transformers version: 3.4.0


## Experiment Settings

In [6]:
total_rounds = 256 # Number of total training rounds
rounds_per_eval = 1 # How often to evaluate
train_clients_per_round = 2 # How many clients to sample per round.
client_epochs_per_round = 1 # Number of epochs in the client to take per round.
batch_size = 20 # Batch size used on the client.
test_batch_size = 100 # Minibatch size of test data.

# Optimizer configuration
server_learning_rate = 1.0 # Server learning rate.
fclient_learning_rate = 0.1 # Client learning rate

## Dataset

In [14]:
train_data, test_data = tff.simulation.datasets.shakespeare.load_data(cache_dir='./tff_shakespeare_cache')

In [21]:
mobilebert_tokenizer = transformers.MobileBertTokenizer.from_pretrained(
    'google/mobilebert-uncased', cache_dir='./transformers_cache')

### Preprocessing

In [20]:
def to_ids(x):
    s = tf.reshape(x['snippets'], shape=[1])
    chars = tf.strings.bytes_split(s).values
    ids = table.lookup(chars)
    return ids

def split_input_target(chunk):
    input_text = tf.map_fn(lambda x: x[:-1], chunk)
    target_text = tf.map_fn(lambda x: x[1:], chunk)
    return (input_text, target_text)

def preprocess(dataset):
    return (
        # Map ASCII chars to int64 indexes using the vocab
        dataset.map(to_ids)
        # Split into individual chars
        .unbatch()
        # Form example sequences of SEQ_LENGTH +1
        .batch(SEQ_LENGTH + 1, drop_remainder=True)
        # Shuffle and form minibatches
        .shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
        # And finally split into (input, target) tuples,
        # each of length SEQ_LENGTH.
        .map(split_input_target))

NameError: name 'vocab' is not defined

In [ ]:
raw_example_dataset = train_data.create_tf_dataset_for_client('THE_TRAGEDY_OF_KING_LEAR_KING')
example_dataset = preprocess(raw_example_dataset)
print(example_dataset.element_spec)

## Model

In [9]:
def tff_model_fn():
    """Constructs a fully initialized model for use in federated averaging."""
    keras_model = transformers.TFMobileBertForMaskedLM.from_pretrained(
        'google/mobilebert-uncased', cache_dir='./transformers_cache')

    loss = tf.keras.losses.SparseCategoricalCrossentropy()

    return simple_fedavg_tf.KerasModelWrapper(keras_model, test_data[0].element_spec, loss)

model = tff_model_fn()

Some layers from the model checkpoint at google/mobilebert-uncased were not used when initializing TFMobileBertForMaskedLM: ['seq_relationship___cls', 'predictions___cls']
- This IS expected if you are initializing TFMobileBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFMobileBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFMobileBertForMaskedLM were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['mlm___cls']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: 'HDF5ClientData' object is not subscriptable

## Training

In [ ]:
iterative_process = simple_fedavg_tff.build_federated_averaging_process(
    tff_model_fn, server_optimizer_fn, client_optimizer_fn)

server_state = iterative_process.initialize()

metric = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [ ]:
for round_num in range(total_rounds):
    sampled_clients = np.random.choice(
        train_data.client_ids,
        size=train_clients_per_round,
        replace=False)

    sampled_train_data = [
        train_data.create_tf_dataset_for_client(client)
        for client in sampled_clients
    ]

    server_state, train_metrics = iterative_process.next(server_state, sampled_train_data)

    print(f'Round {round_num} training loss: {train_metrics}')

    if round_num % rounds_per_eval == 0:
        model.from_weights(server_state.model_weights)

        accuracy = simple_fedavg_tf.keras_evaluate(model.keras_model, test_data, metric)

        print(f'Round {round_num} validation accuracy: {accuracy * 100.0}')